# AMSC698K Homework 3
##### Elijah Kin & Noorain Noorani

### Create a GHZ state of 8 qubits using QISkit or Cirq. Obtain frequency plots for 10,000 samples when...

a) sampling on a QISkit AER simulator or Cirq Simulator without noise model and with the simple noise models for Aria-1 and Forte-1 (show all results on a single frequency plot);

In [ ]:
# TODO

b) sampling on the "ionq_simulator" backend for ideal quantum devices and with noise models "harmony" and "aria-1" (show all results on a single frequency plot);

In [ ]:
# TODO

c) sampling on a real quantum computer ("ionq_qpu.forte-1") with and without error mitigation.

In [ ]:
# TODO

d) using your IBMQE account, find the least busy IBM quantum device, then i) perform the sampling on this quantum device and ii) simulate the sampling using the QISkit noise model for this quantum device.

In [ ]:
# TODO

### 2. A half adder is a logical circuit that adds 2 bits resulting in 2 one-bit registers, one containing the sum, the other the 'carry' (overflow) bit. A full adder adds 2 input bits as well as a carry bit (eg. from addition of less significant bits). 

Expand the quantum circuit for the half adder presented in class to realize a full adder.

In [ ]:
# TODO